In [1]:
import sys
import os
import warnings
import time

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)
warnings.filterwarnings("ignore")

In [2]:
from src.database.db_connection import *
from src.evaluation.metrics import *
from src.utilities.step1_simplify_fds import *

In [3]:
engine1 = create_engine_for_db("primary")
engine2 = create_engine_for_db("secondary")

In [4]:
query_file_name_expected_Holo = "../../../queries/FP/holoclean/hospital_truth.sql"
query_file_name_actual_Holo = "../../../queries/FP/holoclean/hospital_Holo.sql"
algorithm_name = 'Holo'
old_table_name = ''
new_table_name = ''

In [5]:
tids_query = text("""
    SELECT DISTINCT tid
    FROM prototype_fp.hospital_clean
    ORDER BY tid
""")

with engine1.connect() as connection:
    tids = connection.execute(tids_query).scalars().all()

In [6]:
total_precision_Holo, total_recall_Holo,total_coverage_Holo, total_noise_Holo = 0.0, 0.0, 0.0, 0.0
for tid in tids:
    parameters = {"tid": tid}
    results_Holo,results_with_truth_set_Holo = calculate_expected_values(query_file_name_expected_Holo, query_file_name_actual_Holo, algorithm_name,
                                                                             engine1,old_table_name,new_table_name,engine2,parameters)
    precision_Holo, recall_Holo,coverage_Holo, noise_Holo = summarize_results(results_Holo)
    total_precision_Holo += precision_Holo
    total_recall_Holo += recall_Holo
    total_coverage_Holo += coverage_Holo
    total_noise_Holo += noise_Holo

avg_precision_Holo = total_precision_Holo / len(tids)
avg_recall_Holo = total_recall_Holo / len(tids)
avg_coverage_Holo = total_coverage_Holo / len(tids)
avg_noise_Holo = total_noise_Holo / len(tids)

{(57, '10005', 'marshall medical center south', '2505 u s highway 431 north', 'boaz', 'al', '35957', 'marshall', '2565938310', 'acute care hospitals', 'government - hospital district or authority')}
{(57, '10005', 'marshall medical center south', '2505 u s highway 431 north', 'boaz', 'al', '35957', 'marshall', '2x6x938310', 'acute care hospitals', 'government - hospital district or authority')}
0
1
select t."_tid_",t."ProviderNumber" , t."HospitalName" ,t."Address1" ,t."City" ,t."State" ,t."ZipCode", t."CountyName" , t."PhoneNumber", t."HospitalType", t."HospitalOwner" from hospital_repaired  t where t."_tid_"  = :tid;
{(216, '10011', 'st vincents east', '50 medical park east drive', 'birmingham', 'al', '35235', 'jeffersxn', '2058383122', 'acute care hospitals', 'voluntary non-profit - private')}
{(216, '10011', 'st vincents east', '50 medical park east drive', 'birmingham', 'al', '35235', 'jeffersxn', '2058383122', 'acute care hospitals', 'volunxary non-profix - privaxe')}
0
1
select 

In [7]:
print(f"Average Precision: {avg_precision_Holo:.4f}")
print(f"Average Recall: {avg_recall_Holo:.4f}")
print(f"Average Coverage: {avg_coverage_Holo:.4f}")
print(f"Average Noise: {avg_noise_Holo:.4f}")

Average Precision: 0.9920
Average Recall: 0.9910
Average Coverage: 0.9910
Average Noise: 0.0080
